In [43]:
from flatten_dict import flatten
import pandas as pd
import pathlib
#from ..components_modules import acct_dict


In [45]:
path_to_datasets = pathlib.Path.cwd().joinpath("datasets")
path_transactions = path_to_datasets.joinpath("transactions.parquet")
df = pd.read_parquet(path_transactions, engine='pyarrow')

In [46]:
cat_1 = 'Direct Materials'

accounts = flatten(acct_dict[cat_1]).values()
pivot(df, accounts, 'Spend', 'Q', 'Kontobeskrivelse')


,Kjøp Spennarmering,Kjøp Sement rapid,Kjøp Slakkarmering,Kjøp Innstøpingsgods,Kjøp Pukk,Kjøp Sand,Kjøp Sement standard,Kjøp Armeringsnett
2017-03-31,4.883987e+06,3.676159e+06,2.307368e+06,2.255291e+06,9.758596e+05,1.241681e+06,1.700060e+06,1.020333e+06
2017-06-30,4.921245e+06,2.154224e+06,2.142460e+06,2.160594e+06,8.926360e+05,1.768578e+06,1.369680e+06,2.928941e+04
2017-09-30,3.973725e+06,2.702213e+06,8.887682e+05,3.025291e+06,1.676948e+06,2.818162e+06,1.131478e+06,4.943570e+05
2017-12-31,3.635730e+06,2.597736e+06,2.401171e+06,1.941047e+06,1.470595e+06,9.801304e+05,1.270577e+06,9.518498e+05
2018-03-31,3.788388e+06,3.213295e+06,2.156023e+06,1.907110e+06,9.210596e+05,8.157802e+05,1.002056e+06,1.522588e+06
2018-06-30,5.132885e+06,2.630953e+06,2.752831e+06,2.317411e+06,3.358820e+06,8.919875e+05,7.131519e+05,8.039315e+05
2018-09-30,3.039296e+06,3.765868e+06,2.657574e+06,1.371927e+06,5.468008e+06,5.361827e+05,1.482284e+06,9.665740e+05
2018-12-31,4.140215e+06,2.752754e+06,2.160868e+06,1.640927e+06,3.267292e+06,9.478182e+05,1.788218e+06,8.996166e+05
2019-03-31,5.272883e+06,4.745382e+06,1.783541e+06,2.072882e+06,1.651323e+06,1.392943e+06,1.183547e+06,8.290983e+05
2019-06-30,1.938928e+06,4.472961e+06,2.473421e+06,2.475734e+06,9.157294e+05,9.753913e+05,1.492066e+06,1.339064e+06


In [39]:
def pivot(df, accounts, show, frequency, column):
    """
    Filters a dataframe by one or more accounts and
    returns a pivot table of periodic spend,
    number of invoices or number of suppliers.
    The pivot table gets returned as a dictionary
    but is passed to function `transpose_sort_delete`
    where it is turned into a DataFrame and sorted by
    descending values.

    :param df: DataFrame to filter
    :type df: DataFrame
    :param accounts: the accounts to filter by
    :type accounts: int, list
    :param show: the metric to show (spend, invoices, supplier count)
    :type show: str
    :param frequency: weekly, monthly, yearly aggregation
    :type frequency: str
    :param column: the dataframe column to base the pivot_table on
    :type column: str
    :return: returns a DataFrame
    :rtype: DataFrame
    """

    accounts = [accounts] if isinstance(accounts, int) else accounts
    data = df.loc[df['Konto'].isin(accounts)]

    if 'Spend' in show:
        data = data.pivot_table(
            columns=column,
            index=pd.Grouper(freq=frequency, key='Bilagsdato'),
            values='Sum', aggfunc='sum').to_dict()

    elif 'Faktura' in show:
        data = (data
                .drop_duplicates(
                    subset=['Partner', 'Referansenummer'])
                .pivot_table(
                    columns=column,
                    index=pd.Grouper(freq=frequency, key='Bilagsdato'),
                    values='Partner', aggfunc='count').to_dict())

    else:
        data = data.pivot_table(
            columns=column,
            index=pd.Grouper(freq=frequency, key='Bilagsdato'),
            values='Partner', aggfunc=pd.Series.nunique).to_dict()

    return transpose_sort_delete(data)

In [38]:
def transpose_sort_delete(data, accounts=None):
    data = pd.DataFrame(data).fillna(0).transpose()
    if accounts is None:
        data['sum'] = data.sum(axis=1)
        data = data.sort_values(by='sum', ascending=False)[:8]
        data = data.drop(columns=['sum'])
        data = data.transpose()
    else:
        data = data.transpose()
        valid_cols = [acct for acct in accounts if acct in data.columns]
        data = data[valid_cols]

    data.index = [str(x.to_pydatetime().date()) for x in data.index]

    return data

In [42]:
acct_dict = {
        'Direct Materials': {
            'Additives': {
                'Kjøp Tilsettingsmaterialer': 4022,
                'Kjøp Andre tilsettingsstoffer': 4031,
                'Kjøp Silica': 4030,
            },
            'Aggregates': {
                'Kjøp Pukk': 4021,
                'Kjøp Sand': 4020,
                'Kjøp Tilslag for overflatesjikt': 4025,
            },
            'Cement': {
                'Kjøp Sement rapid': 4011,
                'Kjøp Sement standard': 4010,
            },
            'Details': {
                'Kjøp Innstøpingsgods': 4230,
                'Kjøp Montasjedeler 1': 4250,
                'Kjøp Innstøpte løftedetaljer': 4220,
                'Montasjedeler til lager': 4545,
                'Kjøp Stål til formbygging': 4320,
            },
            'Insulation': {
                'Kjøp Isolasjon': 4210,
                'Brannisolering': 4636,
            },
            'Other (form oil, etc)': {
                'Kjøp HD-propper': 4217,
                'Kjøp Formolje': 4310,
                'Kjøp Andre hjelpematerialer': 4330,
            },
            'Prestessing Strand': {
                'Kjøp Spennarmering': 4150,
            },
            'Reinforcement': {
                'Kjøp Slakkarmering': 4160,
                'Kjøp Armeringsnett': 4170,
                'kjøp Kapp og bøy': 4161,
                'Vedl.hold prod.bygninger': 6580,
                'Kjøp Stål til formbygging': 4320,
            },
            'Wooden goods': {
                'Kjøp Finer/Trematerialer formbygg.': 4315,
                'Kjøp Trematerialer underlag': 4316,
            },
        },
        'Indirect Materials': {
            'Maintenance & Repair': {
                'Verneutstyr': 6025,
                'Vedl.hold HD-maskiner/utstyr': 6550,
                'Vedl.hold traverskraner': 6520,
                'Vedl.hold blandestasjoner': 6530,
                'Arbeidsklær': 6020,
                'Vedl.hold el-anlegg': 6582,
                'Kjøp Former til spesielle prosjek.': 4321,
                'Elektroder, sveisetråder': 6410,
                'Forbruksart./Småanskaffelser': 6425,
                'Annet større verktøy': 6451,
                'Håndverktøy': 6452,
                'Vedl.hold annet prod.utstyr': 6570,
                'Vedl.hold arbeidsmaskiner': 6595,
                'Leie av maskiner og utstyr produksjon, korttidsleie': 6350,
                'Løfteutstyr': 6460,
                'Vedl.hold VVS-anlegg': 6583,
                'Kappe- og slipeskiver': 6412,
                'Driftsrekvisita': 6420,
                'Vedl.hold conveyer/halvportal': 6540,
                'Vedl.hold former': 6566,
                'Leasing/Leie trucker': 6340,
                'Kjøp Diamantsagblad': 4325,
                'Vedl.hold spennutstyr': 6562,
                'Rens arbeidsklær': 6030,
                'Fabrikkinventar': 6480,
                'Spennhylser': 6411,
                'Vedl.hold løfteutstyr': 6560,
                'Montasjeverktøy': 6470,
                'Vedl.hold fabrikkinventar': 6510,
                'El. verktøy': 6450,
                'Service/Vedlh.avtale Trucker': 6594,
                'Vedl.hold transportbaner': 6531,
                'Vedl.hold montasjeutstyr': 6575,
                'Vedl.hold vibrasjonsutstyr': 6565,
                'Diverse utgifter': 7698,
            },
            'Human Resources & Finance': {
                'Travel': {
                    'Reise- og oppholdsutgifter': 7020,
                    'Reise-og opphold produksjon': 5086,
                    'Representasjon fradr.berettiget': 7715,
                },
                'Insurance, Warranties & Bank': {
                    'OTP timelønnede': 5081,
                    'AFP - dir. lønn': 5425,
                    'Personalforsikringer': 7520,
                    'Skadeforsikringer': 7510,
                    'AFP - ind.lønn': 5475,
                    'Bankomk og kortgebyr': 7770,
                    'Garantiprovisjoner': 8180,
                    'Etterskuddsrenter lever.': 8140,
                    'Rentekostnader leasing': 8135,
                    'Overf. lønn andre avd. dir': 5018,
                    'Avgiftsfrie godtgjørelser': 5091,
                    'LO/NHO-ordningen - ind. lønn': 5470,
                    'Diverse utgifter': 7690,
                    'Øvrige rentekostnader': 8150,
                },
                'Other (beverage, celebrations etc.)': {
                    'Øvrige velferdstiltak': 7190,
                    'Interne møtekostnader': 7030,
                    'Valgfrie arrangement': 7183,
                    'Treningsavgift': 7191,
                    'Jubileer/Gaver fradragsber.': 7180,
                    'Hytteleie': 7185,
                    'Kantineutstyr': 6016,
                    'Service/vedlh. drikkeautomater': 6018,
                    'Leasing/leie drikkeautomater': 6017,
                    'Bedriftsidrett': 7120,
                    'Yrkesskade pers/utstyr': 7115,
                },
                'Car Leasing': {
                    'Leasingleie firmabiler': 7013,
                    'Driftsutg. person-og varebiler (husk bær': 7010,
                    'Vedlikehold Firmabiler': 7011,
                    'Bompenger og parkering': 7025,
                    'Driftsutgifter varebiler': 4590,
                    'Vedlikehold Varebiler': 4591,
                    'Leasingleie Varebiler': 4593,
                    'Forsikring og avgifter Firmabiler': 7012,
                    'Forsikring og avgifter Varebiler': 4592,
                },
                'Healthcare': {
                    'Bedriftshelsetjenester': 7110,
                },
                'Recruitment & Development': {
                    'Rekrutteringskostnader': 7170,
                    'Kursavgifter': 7090,
                    'Gaver ikke fradragsber.': 7730,
                    'Utg. ved friv. opplæring': 7660,
                    'Diverse utgifter': 7690,
                    'Øresavrunding': 7691,
                },
                'Temp Staf Factories': {
                    'Innleide timer': 5085,
                    'Innleide timer ind.lønn': 5184,
                },
                'Reorganizing Costs': {
                    'Leasing och hyra omorganisering': 7698,
                    'Diverse utgifter omorganisering': 7699,
                },
                'Energy': {
                    'Elektrisk kraft - ordinær': 4370,
                    'Bioenergi': 4377,
                    'Fyringsolje': 4375,
                    'Naturgass': 4376,
                    'Elektrisk kraft - adm.bygg': 6110,
                    'Diesel til arb.maskiner': 6125,
                    'Elektrisk kraft - tilfeldig': 4371,
                },
                'Waste Management': {
                    'Avfallshåndtering': 6255,
                    'Slamsuging': 6257,
                    'Utgifter ved miljøtiltak': 6259,
                },
                'Security': {
                    'Vakthold': 6230,
                },
                'Cleaning': {
                    'Renhold': 6240,
                },
                'Building Maintenance': {
                    'Vedl.hold prod.bygninger': 6580,
                    'Andre eiendomsutgifter': 6265,
                    'Vedl.hold utendørs-anlegg': 6590,
                    'Vedl.hold adm.bygninger': 6610,
                },
                'Office Consumables': {
                    'Kontorrekvisita': 6720,
                    'Kontorutstyr': 6710,
                    'Porto': 6790,
                    'Kop.papir tegninger': 6721,
                },
                'Public Taxes': {
                    'OTP månedslønnede': 7540,
                    'Eiendomsavgifter': 6250,
                    'Festeavgifter': 6380,
                    'Diverse utgifter': 7690,
                    'Andre driftsinntekter avg.fri': 3861,
                },
                'Rent': {
                    'Leie av lokaler': 6210,
                    'Andre lokalkostnader': 6219,
                    'Leie bolig ansatte': 6040,
                },
                'Cafeteria, Food': {
                    'Kantinedrift': 6015,
                    'Mat v/overtid': 6010,
                },
            },
            'IT, Marketing, Consultants': {
                'IT & Communication': {
                    'Vedlikeholdsavtaler Edb': 6880,
                    'Mobiltelefonutg.': 6780,
                    'Vedl.hold printere (inkl serviceavtaler': 6780,
                    'LO/NHO-ordningen dir. lønn': 5420,
                    'Hardware-anskaffelser': 6885,
                    'Leasingkostnader Edb': 6875,
                    'Mobiltelefoner m.utstyr': 6715,
                    'Telefonutgifter': 6770,
                    'Konsulenttjenester Edb': 6890,
                    'Opplæring/Kurs Edb': 6895,
                    'Software-anskaffelser': 6886,
                    'Telefonutg. ansatte': 6775,
                    'Annet vedlikehold Edb': 6881,
                    'Kommun.utstyr (ikke mobil)': 6716,
                    'Linjekostnader Edb': 6870,
                },
                'Consultant Services': {
                    'Fees exceptional for general support services': 7775,
                    'Diverse honorar': 7220,
                    'Konsulenttjenester extern': 7229,
                    'Advokathonorar': 7215,
                    'Revisjonshonorar': 7210,
                    'Konsulenttj./Ark.hon./Prosj.le': 4650,
                    'Diverse utgifter': 7690,
                },
                'Marketing': {
                    'Reklameannonser': 7310,
                    'Aviser, Tidsskrift, abon.m.': 6760,
                    'Sponsorstøtte': 7314,
                    'Reklamemateriell': 7315,
                    'Reklameannonser på interne': 7312,
                    'Diverse markedsføring': 7350,
                    'Trykksaker': 6750,
                    'Byggeplasskilt': 7316,
                    'WEB vedlikehold www.spenncon.no': 7335,
                },
                'Logistics & Transportation': {
                    'Diverse fraktkostnader': 4475,
                    'Frakt båt': 4440,
                    'Utgifter ved mellomlagring': 4445,
                    'Frakt': 4860,
                    'Speditør/Varetaxi': 4450,
                    'Containerleie': 6355,
                },
            },
        },
        'Logistics': {
            'Element Transportation': {
                'Frakt leide biler': 4420,
                'Ventetid ved transport': 4465,
            },
            'Stock Yard (handling & loading)': {
                'Interntransport': 6256,
                'Kranbanetjenester': 4460,
            },
        },
        'Projects': {
            'Subcontractors': {
                'Montasje ved utenforstående': 4510,
                'Stålarbeider ved andre': 4620,
                'Elementkjøp': 4630,
                'Montasje ved utenforstående - timebasert': 4509,
                'Tilleggsarb. v/utenforstående': 4512,
                'Diverse fremmedytelser': 4640,
                'Montasjedeler 1': 4550,
                'Montasjeutg. - materialer': 4530,
                'Hjelpemateriell montasje': 4531,
                'Fuging': 4611,
                'Underentreprenører': 4610,
                'Reise-og opphold v/montasje': 4540,
                'Ståldeler': 4555,
                'Fugemateriell': 4525,
                'Leie øvrig montasjeutstyr': 4516,
            },
            'Services': {
                'Leie-konstuktører-eksterne': 4905,
                'Innleide timer CES': 5084,
            },
            'Rental Equipment': {
                'Leie av krantjenester': 4511,
                'Leie av lift': 4515,
                'Rigg (4502)': 4502,
                'Driftsutgifter mobilkraner': 4580,
            },
            'Assemby Materials': {
                'Betong ved montasje': 4520,
                'Kjøp Ferdigbetong': 4095,
            },
        },
    }